In [1]:
import os
import hail as hl
import pyspark
import bokeh
import logging
import random
import pandas as pd
import numpy as np
from scipy import stats
import pickle 
from matplotlib import pyplot as plt
from typing import Any, Counter, List, Optional, Tuple, Union,Dict,Set
from hail.plot import show, output_notebook
from bokeh.palettes import d3  # pylint: disable=no-name-in-module
from bokeh.models import Plot, Row, Span, NumeralTickFormatter, LabelSet
from gnomad.utils.plotting import *
from typing import Set, Tuple

tmp_dir = "hdfs://spark-master:9820/"
temp_dir = "file:///home/ubuntu/data/tmp"
plot_dir = "/home/ubuntu/data/tmp"

sc = pyspark.SparkContext()
hadoop_config = sc._jsc.hadoopConfiguration()
hadoop_config.set("fs.s3a.access.key", "8YY584J59H7Q6AVKHSU8")
hadoop_config.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_config.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hl.init(sc=sc, tmp_dir=tmp_dir, default_reference='GRCh38')
output_notebook()
logging.basicConfig(format="%(levelname)s (%(name)s %(lineno)s): %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 2.4.7
SparkUI available at http://spark-master:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.60-de1845e1c2f6
LOGGING: writing to /home/ubuntu/data/tmp/scripts/sanger_gnomad_hail_qc/notebooks/hail-20210108-1024-0.2.60-de1845e1c2f6.log


Loading BokehJS ...

In [75]:
run_hash="91b132aa"

ht= hl.read_table(
            f'{temp_dir}/ddd-elgh-ukbb/variant_qc/models/{run_hash}/{run_hash}_rf_result_ranked_denovo_ddd_comp.ht')

mt=hl.read_matrix_table(f'{temp_dir}/ddd-elgh-ukbb/Sanger_cohorts_chr1-7and20_split_sampleqc_filtered.mt')

In [23]:
mt.count()

(13651809, 93307)

9.342915653156297

In [18]:
mt.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        AS_BaseQRankSum: array<float64>, 
        AS_FS: array<float64>, 
        AS_InbreedingCoeff: array<float64>, 
        AS_MQ: array<float64>, 
        AS_MQRankSum: array<float64>, 
        AS_QD: array<float64>, 
        AS_ReadPosRankSum: array<float64>, 
        AS_SOR: array<float64>, 
        BaseQRankSum: float64, 
        DB: bool, 
        DP: int32, 
        DS: bool, 
        END: int32, 
        ExcessHet: float64, 
        FS: float64, 
        InbreedingCoeff: float64, 
        MLEAC: array<int32>, 
        MLEAF: array<float64>, 
        MQ: float64, 
        MQRankSum: floa

In [76]:
#mt1=mt.annotate_rows(info_rf_probability=ht[mt.row_key].rf_probability['TP'])
mt=mt.annotate_rows(
    info = mt.info.annotate(rf_probability=ht[mt.row_key].rf_probability['TP'])
)
mt=mt.annotate_rows(
    info = mt.info.annotate(score=ht[mt.row_key].score)
)

In [20]:
mt.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        AS_BaseQRankSum: array<float64>, 
        AS_FS: array<float64>, 
        AS_InbreedingCoeff: array<float64>, 
        AS_MQ: array<float64>, 
        AS_MQRankSum: array<float64>, 
        AS_QD: array<float64>, 
        AS_ReadPosRankSum: array<float64>, 
        AS_SOR: array<float64>, 
        BaseQRankSum: float64, 
        DB: bool, 
        DP: int32, 
        DS: bool, 
        END: int32, 
        ExcessHet: float64, 
        FS: float64, 
        InbreedingCoeff: float64, 
        MLEAC: array<int32>, 
        MLEAF: array<float64>, 
        MQ: float64, 
        MQRankSum: floa

In [21]:
mt.info.show()

+---------------+---------------+--------------+----------------+---------+----------------------+----------------+
| locus         | alleles       | info.AC      | info.AF        | info.AN | info.AS_BaseQRankSum | info.AS_FS     |
+---------------+---------------+--------------+----------------+---------+----------------------+----------------+
| locus<GRCh38> | array<str>    | array<int32> | array<float64> |   int32 | array<float64>       | array<float64> |
+---------------+---------------+--------------+----------------+---------+----------------------+----------------+
| chr1:12938    | ["GCAAA","G"] | [2]          | [3.43e-04]     |    5828 | NA                   | NA             |
| chr1:13024    | ["G","A"]     | [2]          | [1.15e-04]     |   17384 | NA                   | NA             |
| chr1:13087    | ["A","G"]     | [5]          | [1.24e-04]     |   40192 | NA                   | NA             |
| chr1:13116    | ["T","C"]     | [2]          | [4.91e-05]     |   40732 | NA                   | NA             |
| chr1:13130    | ["C","T"]     | [2]          | [5.57e-05]     |   35920 | NA                   | NA             |
| chr1:13151    | ["G","C"]     | [2]          | [4.80e-05]     |   41646 | NA                   | NA             |
| chr1:13164    | ["G","A"]     | [2]          | [4.30e-05]     |   46532 | NA                   | NA             |
| chr1:13176    | ["G","T"]     | [3]          | [5.93e-05]     |   50596 | NA                   | NA             |
| chr1:13198    | ["C","A"]     | [1]          | [1.72e-05]     |   58304 | NA                   | NA             |
| chr1:13216    | ["C","G"]     | [5]          | [7.96e-05]     |   62810 | NA                   | NA             |
+---------------+---------------+--------------+----------------+---------+----------------------+----------------+

+-------------------------+----------------+-------------------+----------------+------------------------+----------------+
| info.AS_InbreedingCoeff | info.AS_MQ     | info.AS_MQRankSum | info.AS_QD     | info.AS_ReadPosRankSum | info.AS_SOR    |
+-------------------------+----------------+-------------------+----------------+------------------------+----------------+
| array<float64>          | array<float64> | array<float64>    | array<float64> | array<float64>         | array<float64> |
+-------------------------+----------------+-------------------+----------------+------------------------+----------------+
| [-1.68e-01]             | NA             | NA                | [2.54e+01]     | NA                     | NA             |
| [-1.47e-01]             | NA             | NA                | [3.10e+01]     | NA                     | NA             |
| [-1.26e-01]             | NA             | NA                | [7.23e+00]     | NA                     | NA             |
| [-1.32e-01]             | NA             | NA                | [2.46e+01]     | NA                     | NA             |
| [-1.47e-01]             | NA             | NA                | [2.74e+01]     | NA                     | NA             |
| [-1.38e-01]             | NA             | NA                | [1.65e+01]     | NA                     | NA             |
| [-1.29e-01]             | NA             | NA                | [1.95e+01]     | NA                     | NA             |
| [-1.20e-01]             | NA             | NA                | [1.12e+01]     | NA                     | NA             |
| [-1.01e-01]             | NA             | NA                | [3.80e+00]     | NA                     | NA             |
| [-8.65e-02]             | NA             | NA                | [1.07e+01]     | NA                     | NA             |
+-------------------------+----------------+-------------------+----------------+------------------------+----------------+

+-------------------+---------+---------+---------+----------+----------------+----------+----------------------+--------------+
| info.BaseQRankSum | info.DB

In [22]:
mt.filters.show()

,,
locus,alleles,filters
locus<GRCh38>,array<str>,set<str>
chr1:12938,"[""GCAAA"",""G""]",NA
chr1:13024,"[""G"",""A""]",NA
chr1:13087,"[""A"",""G""]",NA
chr1:13116,"[""T"",""C""]",NA
chr1:13130,"[""C"",""T""]",NA
chr1:13151,"[""G"",""C""]",NA
chr1:13164,"[""G"",""A""]",NA
chr1:13176,"[""G"",""T""]",NA


In [77]:
mt = mt.annotate_rows(
                Variant_Type=hl.cond((hl.is_snp(mt.alleles[0], mt.alleles[1])), "SNP",
                                     hl.cond(
                                         hl.is_insertion(mt.alleles[0], mt.alleles[1]),
                                         "INDEL",
                                         hl.cond(hl.is_deletion(mt.alleles[0],
                                                                mt.alleles[1]), "INDEL",
                                                 "Other"))))

/home/ubuntu/venv/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated function (or staticmethod) cond. (Replaced by hl.if_else) -- Deprecated since version 0.2.59.
  


In [78]:
#mt1=mt.annotate_rows(filternew=(hl.case()
#                           .when( (hl.is_snp(
#        mt.alleles[0], mt.alleles[1])) & (mt.info.rf_probability <=90) , "PASS")))


filter_column_annotation = (
                hl.case()
                    .when(((mt.Variant_Type == "SNP") & (mt.info.rf_probability <=0.90)), "PASS")
                    .when(((mt.Variant_Type == "INDEL") & (mt.info.rf_probability <=0.80)), "PASS")
                    .default(".")  # remove everything else
            )

#mt_annotated = mt.annotate_rows(mt.filters=filter_column_annotation)
mt1=mt.annotate_rows(
    filtercol = ((filter_column_annotation))
)
#hl.literal(set(variants_to_include.key.locus.collect()))

In [64]:
mt1.filtercol.summarize()

Non-missing,13650282 (99.99%)
Missing,1527 (0.01%)
Min Size,4
Max Size,4
Mean Size,4.00
Sample Values,"['PASS', 'PASS', 'PASS', 'PASS', 'PASS']"


In [79]:
mt_fail=mt1.filter_rows(mt1.filtercol==".")

In [80]:
mt_fail.count()

(1055888, 93307)

In [69]:
(1275477/13651809) *100

9.342915653156297

In [73]:
(1055888/13651809)*100

7.734418200547634

In [88]:
mt2=mt1.annotate_rows(filters=mt1.filters.add(mt1.filtercol))

In [89]:
mt2.filters.show()

,,
locus,alleles,filters
locus<GRCh38>,array<str>,set<str>
chr1:12938,"[""GCAAA"",""G""]","{""PASS""}"
chr1:13024,"[""G"",""A""]","{""PASS""}"
chr1:13087,"[""A"",""G""]","{""PASS""}"
chr1:13116,"[""T"",""C""]","{""PASS""}"
chr1:13130,"[""C"",""T""]","{""PASS""}"
chr1:13151,"[""G"",""C""]","{""PASS""}"
chr1:13164,"[""G"",""A""]","{""PASS""}"
chr1:13176,"[""G"",""T""]","{""PASS""}"


In [92]:
mt_fail2=mt2.filter_rows(mt2.filters.contains("."))

In [93]:
mt_fail2.count()

(1055888, 93307)

In [94]:
mt_pass=mt2.filter_rows(mt2.filters.contains("PASS"))

In [95]:
mt_pass.count()

(12594394, 93307)